In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [ ]:
import gif
from edesdetectrl.environments.m_mode_binary_classification.line import (
    Bounds,
    Line,
    MModeLine,
)
from jax import random
import matplotlib.pyplot as plt

key = random.PRNGKey(42)
bounds = Bounds(-40, -40, 40, 40)
lines = []


def next_random_line():
    global key, bounds, lines
    lines.append(Line.random_within_bounds(key, bounds, 55, pad_x=10, pad_y=10))
    key, = random.split(key, 1)
    return lines[-1]


In [ ]:
import matplotlib.patches as patches


@gif.frame
def frame():
    fig, ax = plt.subplots(figsize=(6, 6))
    bounds2 = Bounds(-50, -50, 50, 50)

    [next_random_line() for _ in range(99)]
    ax.imshow(
        MModeLine(next_random_line(), bounds2, 200).visualize_line(),
        extent=[-50, 50, -50, 50],
        aspect="auto",
    )
    ax.scatter(
        [(line.x1 + line.x2) / 2 for line in lines],
        [(line.y1 + line.y2) / 2 for line in lines],
    )
    ax.add_patch(
        patches.Rectangle((-40, -40), 80, 80, fc="none", ec="w", lw=1, ls="dashed")
    )
    ax.add_patch(
        patches.Rectangle((-30, -30), 60, 60, fc="none", ec="g", lw=1, ls="dashed")
    )
    return ax


gif.save([frame() for _ in range(200)], "temp.gif")

In [ ]:
import numpy as np

a, b = np.histogram([line.rotation for line in lines], 24)
plt.plot(b[:-1],a)
None

In [ ]:
a, b, c = np.histogram2d(
    [(line.x1 + line.x2) / 2 for line in lines],
    [(line.y1 + line.y2) / 2 for line in lines],
    bins=32
)

plt.hist2d(
    [(line.x1 + line.x2) / 2 for line in lines],
    [(line.y1 + line.y2) / 2 for line in lines],
    c,
)
None

In [ ]:
plt.plot(np.sqrt(np.sum(a, axis=0)*np.sum(a, axis=1)))